In [1]:
import requests
from bs4 import BeautifulSoup
import time

import pandas as pd
import numpy as np

In [2]:
def get_text_from_genius_of(author, title):
    author = author.replace(" ", "-")
    title = title.replace(" ", "-")
    URL = "https://genius.com/" + author + "-" + title + "-lyrics"
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")
    
    lyrics = ""
    
    lyrics_containers = soup.find_all("div", {"data-lyrics-container" : "true"})
    
    if len(lyrics_containers) == 0:
        return None
    
    for lyrics_container in lyrics_containers:
        for br in lyrics_container.find_all("br"):
            br.replace_with("\n")
        lyrics += lyrics_container.text
    
    next_brackets = lyrics.find("[")
    while next_brackets != -1:
        lyrics = lyrics[:next_brackets] + lyrics[lyrics.find("]")+1:]
        next_brackets = lyrics.find("[")
    
    lyrics = lyrics.replace("\n\n", "\n").replace("\n", " ")
    return lyrics

In [3]:
songsdata_df = pd.read_csv('batch2.csv')
songsdata_df = songsdata_df[['track', 'artist', 'genre', 'valence_tags', 'arousal_tags']]
display(songsdata_df)

,track,artist,genre,valence_tags,arousal_tags
0,Born & Raised,Fake Problems,punk,7.496667,5.113333
1,Trial By Fire,Kiss,hard rock,3.850000,5.640000
2,Sunrise Calendar,Tycho,ambient,6.990000,3.795000
3,The Flower Called Nowhere,Stereolab,electronica,3.980000,2.957143
4,When I am Laid in Earth (Dido's Lament),Henry Purcell,NaN,3.520000,5.360000
...,...,...,...,...,...
7495,Prince of E-Ville,Combichrist,industrial,4.055508,4.560678
7496,I've Got That Feeling,The Kinks,classic rock,5.000000,4.170000
7497,"Cold, Hard Fact",Overkill,thrash metal,4.320000,3.550000
7498,Into My Arms,Nick Cave & The Bad Seeds,alternative,6.127333,4.384667


In [4]:
def to_alfanum(txt):
    return ''.join(ch for ch in txt if ch.isalnum() or ch == ' ')


lyrics = []
for i, (_, row) in enumerate(songsdata_df.iterrows()):
    track = to_alfanum(row['track'])
    author = to_alfanum(row['artist'])
    text = get_text_from_genius_of(author, track)
    lyrics.append(text)
    if i % 1000 == 0:
        print(i)

0
1000
2000


MemoryError: 

In [7]:
songsdata_df = songsdata_df.iloc[:2642, : ]
songsdata_df['lyrics'] = lyrics

<ipython-input-7-8c5d99fcc7f1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  songsdata_df['lyrics'] = lyrics


In [8]:
songsdata_df['lyrics'] = lyrics

<ipython-input-9-fc162a9eb32a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  songsdata_df['lyrics'] = lyrics


In [9]:
songsdata_df.to_csv('lyrics2.csv')

In [11]:
display(songsdata_df)

,track,artist,genre,valence_tags,arousal_tags,lyrics
1,Trial By Fire,Kiss,hard rock,3.850000,5.640000,"Only got one life to live, I'm going to live ..."
2,Sunrise Calendar,Tycho,ambient,6.990000,3.795000,Houses and fields are especially Plants into ...
3,The Flower Called Nowhere,Stereolab,electronica,3.980000,2.957143,All the small boats on the water Aren't going...
5,I'll Be Waiting,Eurielle,noise,5.510588,4.205294,If I win or I lose Doesn't matter 'cause I fe...
8,Full of Fire,The Knife,experimental,3.995000,3.745000,Sometimes I get problems that are hard to solv...
...,...,...,...,...,...,...
2631,Hit the Ground,Zebrahead,rock,7.570000,6.100000,(Anesthetized by these lies) (I don't believe ...
2633,Wholly Night,Arsis,melodic death metal,5.000000,5.280000,As the moon creeps forth to pierce the clouds ...
2634,"Yes, Even Stars Break",The Scene Aesthetic,acoustic,6.768519,4.729444,Alone at last Sleeping to help ease my pain An...
2635,S.O.S.,Take That,pop,6.070833,4.714167,Save our souls we're splitting atoms Go tell ...
